In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 👉 Install Pycaret

In [ ]:
!pip install pycaret

# 👉 Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import * 

# 👉 Loading Train and Test dataset

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
test_df.head()

# 👉 Making a Copy of train_df dataframe

In [ ]:
train_df_copy = train_df.copy()

# 👉 Dropping 'Survived' column from train dataset

In [ ]:
train_df.drop('Survived', axis=1, inplace = True)

# 👉 Concatenate train_df and test_df dataframe

In [ ]:
df = pd.concat([train_df,test_df], ignore_index=True)
df.tail()

# Data Preprocessing
## 👉 Check for null values

In [ ]:
df.isnull().sum()

## 👉 Check feature information

In [ ]:
df.info()

## 👉 Check feature description

In [ ]:
df.describe()

## 👉 Visualizing the features

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(df['Age'], bins = 20)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Histogram of Age')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(df['Fare'], bins = 50)
plt.xlabel('Fare')
plt.ylabel('Count')
plt.title('Histogram of Fare')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(df['SibSp'], bins = 20)
plt.xlabel('SibSp')
plt.ylabel('Count')
plt.title('Histogram of SibSp')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(df['Parch'], bins = 20)
plt.xlabel('Parch')
plt.ylabel('Count')
plt.title('Histogram of Parch')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(df['Pclass'], bins = 5)
plt.xlabel('Pclass')
plt.ylabel('Count')
plt.title('Histogram of Parch')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(x="Embarked", data=df)

## 👉 Imputing null values for all the features

In [ ]:
age_mean = df['Age'].mean()
age_median = df['Age'].median()
print(age_mean, age_median)

In [ ]:
df['Age'] = df['Age'].fillna(age_median)

In [ ]:
fare_mean = df['Fare'].mean()
fare_median = df['Fare'].median()
print(fare_mean, fare_median)

In [ ]:
df['Fare'] = df['Fare'].fillna(fare_median)

In [ ]:
embarked_mode = df['Embarked'].mode()[0]
print(embarked_mode)

In [ ]:
df['Embarked'] = df['Embarked'].fillna(embarked_mode)

In [ ]:
df['Ticket'] = df['Ticket'].fillna('NA')

In [ ]:
df['Cabin'] = df['Cabin'].fillna('NA')

## 👉 Once again verifying the null values

In [ ]:
df.isnull().sum()

# Feature Engineering
## 👉 Creating new features 'First_Name' and 'Last_Name'

In [ ]:
df[['First_Name','Last_Name']] = df.Name.str.split(',', expand=True)
df.head()

## 👉 Creating new feature 'Family'

In [ ]:
df['Family'] = df['SibSp'] + df['Parch'] +1

## 👉 Creating new feature 'Cabin_Class'

In [ ]:
df['Cabin_Class'] = df['Cabin'].str[:1]

In [ ]:
df['Cabin_Class'] = df['Cabin_Class'].replace(to_replace = 'N', value = 'NA')

## 👉 Dropping 'Name' and 'PassengerId' columns from df

In [ ]:
df.drop(['Name','PassengerId'], axis=1, inplace = True)

In [ ]:
df.head()

## 👉 Rounding the 'Age' and 'Fare' columns

In [ ]:
#df['Age'] = df['Age'].round().astype('int')
df['Age'] = df['Age'].round()

In [ ]:
#df['Fare'] = df['Fare'].round().astype('int')
df['Fare'] = df['Fare'].round()

In [ ]:
df.head()

## 👉 Perform Scaling, Label encoding and One hot encoding on categorical features

In [ ]:
label = LabelEncoder()

In [ ]:
df['Ticket'] = label.fit_transform(df['Ticket'])
df['First_Name'] = label.fit_transform(df['First_Name'])
df['Last_Name'] = label.fit_transform(df['Last_Name'])
df['Cabin_Class'] = label.fit_transform(df['Cabin_Class'])
df['Cabin'] = label.fit_transform(df['Cabin'])

In [ ]:
df_onehot = pd.get_dummies(df[['Sex','Embarked']])
df_onehot.head()

In [ ]:
df = pd.concat([df,df_onehot], axis=1)
df.head()

## 👉 Dropping 'Sex' and 'Embarked' columns

In [ ]:
df.drop(['Sex','Embarked'], axis=1, inplace = True)

In [ ]:
df.head()

## 👉 Splitting the Train and Test data

In [ ]:
train = df[0:100000]
test = df[100000:]

## 👉 Appending 'Survived'column to train dataframe

In [ ]:
train = pd.concat([train, train_df_copy['Survived']], axis=1)
train.head()

## 👉 Splitting training data into training and validation dataset

In [ ]:
data = train.sample(frac=0.95, random_state=786)
data_unseen = train.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

## 👉 Setting up preprocessing sequence for Pycaret

In [ ]:
titanic_classification = setup(
                               data= data, 
                               target='Survived', 
                               ignore_features = ['First_Name'],
                               numeric_features =['SibSp','Parch','Family','Age','Fare','Ticket','Cabin','Cabin_Class','Last_Name'], 
                               ordinal_features = {'Pclass': ['1','2','3']},
                           #   high_cardinality_features = ['Ticket'],
                               normalize=True,
                               normalize_method = 'maxabs',
                               handle_unknown_categorical = True,
                               unknown_categorical_method = 'least_frequent',
                            #  bin_numeric_features = ['Age','Fare'],
                               remove_outliers = True,
                            #  remove_multicollinearity = True,
                            #   polynomial_features = True,
                               trigonometry_features = True,
                               transformation= True, 
                               fix_imbalance=True, 
                               use_gpu=True, 
                               session_id=123, 
                               log_experiment=True, 
                               silent=True)

## 👉 Comparing different models

In [ ]:
best_model = compare_models()

## 👉 Choose the best model from the list

In [ ]:
lightgbm = create_model('lightgbm')

## 👉 Tune the model

In [ ]:
tuned_lightgbm = tune_model(lightgbm)

## 👉 Plot the AUC curve

In [ ]:
plot_model(lightgbm, plot = 'auc')

## 👉 Plot Precision-Recall Curve

In [ ]:
plot_model(lightgbm, plot = 'pr')

## 👉 Plot feature Importance

In [ ]:
plot_model(lightgbm, plot='feature')

## 👉 Plot Confusion Matrix

In [ ]:
plot_model(lightgbm, plot = 'confusion_matrix')

## 👉 Evaluating the model

In [ ]:
evaluate_model(lightgbm)

## 👉 Predict data

In [ ]:
predict_model(lightgbm)

## 👉 Finalize Model for deployment

In [ ]:
final_lightgbm = finalize_model(lightgbm)

In [ ]:
print(final_lightgbm)

In [ ]:
predict_model(final_lightgbm)

## 👉 Predict model on unseen data

In [ ]:
unseen_predictions = predict_model(final_lightgbm, data=data_unseen)
unseen_predictions.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['Survived'], unseen_predictions['Label'], metric = 'Accuracy')

## 👉 Saving the model

In [ ]:
save_model(final_lightgbm,'Final lgm Model 26Apr2021')

## 👉 Loading the saved model

In [ ]:
saved_final_lightgbm = load_model('Final lgm Model 26Apr2021')

## 👉 Predicting on test data

In [ ]:
submission = predict_model(saved_final_lightgbm, data=test)
submission.head()

## 👉 Creating a submission file

In [ ]:
submission_copy = submission.copy()

In [ ]:
submission_copy.drop(['Pclass', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Last_Name', 'Sex_female', 'Sex_male', 'Embarked_C',
       'Embarked_Q', 'Embarked_S', 'Score'], axis=1, inplace = True)

In [ ]:
submission_copy.rename(columns={'Label':'Survived'},inplace = True)

In [ ]:
submission_copy.to_csv('submission.csv', index=False)